# Choosing How to Representing Data

_Today we'll revisit how redundant storage of data can improve the efficiency of particular queries. We'll introduce a technique to ensure consistency among the redundant data. We'll then take advantage of our repetitive data to eliminate the need for some storage._

## A Meeting Calendar


The data we'll be working with is a calendar of meetings. Meetings have a title, a date, and a set of invited people. Our users want to add meetings to their calendar with an ***add_meeting*** function. 

Implement Calendar's ***add_meeting*** function below. Let's keep a dictionary keyed by the meeting dates, so that we can query for the meetings on a particular day.

In [22]:
class Meeting():
    def __init__(self, title, date, invitees):
        self.title = title
        self.date = date
        self.invitees = invitees 
            
    ########### ignore this method ###########
    def pprint(self, num_people_to_show=5):
        invitees = ""
        personi = 0
        for person in self.invitees:
            if personi <= num_people_to_show:
                invitees = invitees + person + ", "
                personi = personi + 1
            else:
                invitees = invitees[:-2] + "....."
                break
        result = f"{self.title} with {invitees[:-2]}"
                
        print(result)
        
        
class MeetingGroup():
    def __init__(self, members):
        self.members = members
        # meetings keyed by date
        self.meetings = {}
        
    def add_meeting(self, meeting):
        """
        Add a meeting to this MeetingGroup
        Make sure the meeting.invitees points to the MeetingGroup's members to save space
        """
        meeting.invitees = self.members
        self.meetings.setdefault(meeting.date, []).append(meeting)

    ############## ignore this method ##############
    def pprint(self):
        for date, meetings in self.meetings.items():
            print(date + ": ") 
            for meeting in meetings:
                meeting.pprint()
            print()
        
        
class Calendar():
    def __init__(self):
        # define any data structures needed to keep track of the meetings here
        self.dates = {}
        
        self.people = {}
        
        self._check_rep()
        
        self.groups = {}
        
    def add_meeting(self, meeting):
        """
        Add meeting to the calendar
        """
        self.dates.setdefault(meeting.date, []).append(meeting)
        
        for invitee in meeting.invitees:
            self.people.setdefault(invitee, []).append(meeting)
            
        self._check_rep()
        
        members = frozenset(meeting.invitees)
        #self.groups.setdefault(members, MeetingGroup(members)).add_meeting(meeting)
        if members not in self.groups:
            self.groups[members] = MeetingGroup(members)
        self.groups[members].add_meeting(meeting)

    
    def get_persons_meetings(self, person):
        """
        Return all of a certain person's meetings
        """
        if person in self.people:
            return self.people[person]
        else:
            raise LookupError("This person has no meetings.")
    
    def get_group(self, members):
        """
        Return a MeetingGroup corresponding to the people in members
        """
        members = frozenset(members)
        if members in self.groups:
            return self.groups[members]
        else:
            raise LookupError("This group has not been added to the Calendar")
            
    def _check_rep(self):
        """Ensures that the internal representations of the meeting data
        are consistent; checks that all meetings in the people dictionary
        are also in the date dictionary, and vice versa. If not, raises 
        an AssertionError. """
        
        for dated_meetings in self.dates.values():
            for meeting in dated_meetings:
                for person in meeting.invitees:
                    assert meeting in self.people[person]
        
        for personed_meetings in self.people.values():
            for meeting in personed_meetings:
                assert meeting in self.dates[meeting.date]
                

    ############## ignore this method ##############
    def pprint(self):
        for date, meetings in self.dates.items():
            print(date + ": ") 
            for meeting in meetings:
                meeting.pprint()
            print()

## Add a Bunch of Meetings

Let's practice adding meetings to our calendar.

In [17]:
calendar = Calendar()

In [3]:
meeting_to_add = Meeting("Lecture 9", "04-01-19", {"Srini"})
calendar.add_meeting(meeting_to_add)

In [4]:
calendar.pprint()

04-01-19: 
Lecture 9 with Srini



Let's add a bunch more. (Don't get caught up in these details--lots of code just to add fun example meetings...)

In [5]:
instructors = {"Adam", "Srini", "Eric"}
tas = {"Yaseen", "Angie", "Jeff", "Zachary", "Alexandra", "Rami", "Srijon", 
       "Aron", "Valerie", "Elijah", "Kentaro", "Yanni", "Eyob"}


quiz_team = {"Eyob", "Yanni"}
mon_staff = {"Jeff", "Kentaro", "Srijon"}
tues_staff = {"Angie", "Valerie", "Jeff"}

In [6]:
def fill_cal(calendar):
    calendar.add_meeting(Meeting("Quiz Planning", "10-05-19", quiz_team | instructors))
    calendar.add_meeting(Meeting("Quiz Prep", "10-10-19", quiz_team))

    calendar.add_meeting(Meeting("Lecture Planning", "10-06-19", instructors))
    calendar.add_meeting(Meeting("Staff Meeting", "10-08-19", tas | instructors))

    calendar.add_meeting(Meeting("Monday OHs", "10-07-19", set(mon_staff)))
    calendar.add_meeting(Meeting("Tuesday OHs", "10-08-19", tues_staff))
    calendar.add_meeting(Meeting("Monday OHs", "10-14-19", mon_staff))
    calendar.add_meeting(Meeting("Tuesday OHs", "10-15-19", tues_staff))
    calendar.add_meeting(Meeting("Monday OHs", "10-21-19", mon_staff))
    calendar.add_meeting(Meeting("Tuesday OHs", "10-22-19", tues_staff))
    calendar.add_meeting(Meeting("Monday OHs", "10-28-19", mon_staff))
    calendar.add_meeting(Meeting("Tuesday OHs", "10-29-19", tues_staff))

fill_cal(calendar)

In [7]:
calendar.pprint()

04-01-19: 
Lecture 9 with Srini

10-05-19: 
Quiz Planning with Eric, Srini, Adam, Yanni, Eyob

10-10-19: 
Quiz Prep with Yanni, Eyob

10-06-19: 
Lecture Planning with Eric, Srini, Adam

10-08-19: 
Staff Meeting with Aron, Jeff, Elijah, Srijon, Zachary, Alexandra...
Tuesday OHs with Valerie, Angie, Jeff

10-07-19: 
Monday OHs with Jeff, Kentaro, Srijon

10-14-19: 
Monday OHs with Jeff, Kentaro, Srijon

10-15-19: 
Tuesday OHs with Valerie, Angie, Jeff

10-21-19: 
Monday OHs with Jeff, Kentaro, Srijon

10-22-19: 
Tuesday OHs with Valerie, Angie, Jeff

10-28-19: 
Monday OHs with Jeff, Kentaro, Srijon

10-29-19: 
Tuesday OHs with Valerie, Angie, Jeff



## Adam wants to know his Schedule

Now, Adam wants to know his schedule, so we need to implement a function that queries for this as well. However, with the data representation we currently have, we'd have to loop through all of the dates, then loop through each of the meetings on each of the dates. On large calendars in which any individual is invited to only a small subset of meetings, this would be very inefficient. Let's trade memory use for greater performance on these queries by also storing another dictionary, keyed by people.

In [11]:
calendar = Calendar()
fill_cal(calendar)

adams_meetings = calendar.get_persons_meetings("Adam")

for mtg in adams_meetings:
    mtg.pprint()

Quiz Planning with Eric, Srini, Adam, Yanni, Eyob
Lecture Planning with Eric, Srini, Adam
Staff Meeting with Aron, Jeff, Elijah, Srijon, Zachary, Alexandra...


Now we can tell Adam his meetings much more quickly!

## Ensuring Consistency

We now have two different representations of our meetings. How can we make sure that they remain internally consistent? Let's make a function to check the representation, ***_check_rep***. 

Note the use of the underscore at the beginning of the method name. This is a python convention for functions which aren't meant to be accessed outside of the class. We don't expect clients making Calendars to need to check their internal representation, so we keep this method private to the implementers.

Try breaking the consistency between the two internal data structures yourself, and make sure ***_check_rep*** raises an error. 

The implementation of ***_check_rep*** can be quite inefficient. We can use it for testing our Calendar, but if we were to actually put this Calendar to use, we would most likely remove it in favor of efficiency.

In [13]:
calendar = Calendar()
fill_cal(calendar)
calendar.pprint()

10-05-19: 
Quiz Planning with Eric, Srini, Adam, Yanni, Eyob

10-10-19: 
Quiz Prep with Yanni, Eyob

10-06-19: 
Lecture Planning with Eric, Srini, Adam

10-08-19: 
Staff Meeting with Aron, Jeff, Elijah, Srijon, Zachary, Alexandra...
Tuesday OHs with Valerie, Angie, Jeff

10-07-19: 
Monday OHs with Jeff, Kentaro, Srijon

10-14-19: 
Monday OHs with Jeff, Kentaro, Srijon

10-15-19: 
Tuesday OHs with Valerie, Angie, Jeff

10-21-19: 
Monday OHs with Jeff, Kentaro, Srijon

10-22-19: 
Tuesday OHs with Valerie, Angie, Jeff

10-28-19: 
Monday OHs with Jeff, Kentaro, Srijon

10-29-19: 
Tuesday OHs with Valerie, Angie, Jeff



## Reducing Redundancy

So far, we decided to introduced lots of redundancy by storing meetings twice: once keyed by the dates, and once keyed by the invitee. Now, let's try to win back a bit of storage space.

Note that currently, we could be storing a different set of invitees for each meeting, even if this is redudant with other meetings having the same set of invitees. To save space, let's make sure that we only store each set of invitees once in the calendar.

To do this, we will add another class, MeetingGroup. Each meeting group will have a unique set of invitees, and meetings contained in a MeetingGroup should point their invitees set to their MeetingGroup's set. This way, we will only keep one copy of each unique set of invitees, saving space. In addition, we will be able to efficiently access all of the meetings that a certain group of invitees are invited to. 

Implement MeetingGroup's ***add_meeting*** function above, and make sure the code below runs.

In [19]:
group = MeetingGroup(tues_staff)
meeting1 = Meeting("Monday OHs", "10-07-19", set(tues_staff))
meeting2 = Meeting("Tuesday OHs", "10-08-19", set(tues_staff))

# currently storing three different copies of the members
assert meeting1.invitees is not group.members and \
       meeting2.invitees is not group.members and \
       meeting1.invitees is not meeting2.invitees

group.add_meeting(meeting1)
group.add_meeting(meeting2)

# we should only have only one copy
for date, meetings in group.meetings.items():
    for meeting in meetings:
        assert meeting.invitees is group.members

print("success!")

success!


Notice that once we add the meetings to the group, we no longer have a reference to the two sets we passed into meeting1 and meeting2. These will automatically be freed from memory by Python, so you don't have to worry about them anymore.

Now, add code to ***add_meeting*** and ***__init__*** as well as implement ***get_group*** in Calendar. We should be able to efficiently access the MeetingGroups and add meetings to them in Calendar. Note that we cannot use sets as dictionary keys since they are mutable, so consider using frozensets, the immutable equivalent.

In [23]:
calendar = Calendar()
fill_cal(calendar)
calendar.get_group(mon_staff).pprint()

10-07-19: 
Monday OHs with Jeff, Kentaro, Srijon

10-14-19: 
Monday OHs with Jeff, Kentaro, Srijon

10-21-19: 
Monday OHs with Jeff, Kentaro, Srijon

10-28-19: 
Monday OHs with Jeff, Kentaro, Srijon



## The Takeaways!

- Redundant storage can sometimes improve performance
- Redundant storage introduces the risk of internal inconsistency and weird behavior
    - One solution is to use a ***_check_rep*** function to fail fast whenever inconsistencies arise
- Sometimes data has inherent redundancies which can be exploited to reduce storage space
    - Here, we saw this was a tradeoff with implementation complexity
    
You'll see these ideas in lab 8 and beyond!